In [32]:
import advent
advent.scrape(2016, 11)
from advent.maze import solve_maze
from typing import NamedTuple

class Node(NamedTuple):
    floor: int
    floors: tuple[frozenset[str], frozenset[str], frozenset[str], frozenset[str]]

    def move_item(self, floorto: int, item: str, moveelevator=True) -> 'Node':
        #assert item in self.floors[floorfrom]
        floorfrom = self.floor
        floors = list(self.floors)
        floors[floorfrom] = floors[floorfrom].difference([item])
        floors[floorto] = floors[floorto] | {item}
        return Node(floorto if moveelevator else self.floor,
                    tuple(floors))

e = frozenset([])
items = ['SG', 'SM', 'PG', 'PM', 'TM', 'TG', 'RG', 'RM', 'CG', 'CM']
start = Node(
    0,
    (frozenset(['SG', 'SM', 'PG', 'PM']),
    frozenset(['TG', 'RG', 'RM', 'CG', 'CM']),
    frozenset(['TM']),
    e)
)
# I lost like 15 minutes because I mistyped 4 here... It was almost midnight...
end = (3 , (e, e, e, frozenset(items)))

In [ ]:
from itertools import combinations

def safe(floor: frozenset[str]):
    # it's unsafe if for any M: there is any G, but the G corresponding to the M is not on the floor
    any_g = any(item.endswith('G') for item in floor)
    for item in floor:
        if item.endswith('G'): continue
        elif f"{item[0]}G" in floor: continue
        elif any_g: return False
    return True

def elevator_directions(i: int):
    if i == 0: return [1]
    if i == 1: return [0, 2]
    if i == 2: return [1, 3]
    if i == 3: return [2]
    raise ValueError()

# Well this is a fun one...
# My first thought was Dijkstra, let's try it
# As for the input parsing, it seems so much easier to do it manually
def adjacent(node: Node):
    assert node.floor >= 0 and node.floor < 4
    flooritems = node.floors[node.floor]
    if len(flooritems) == 0: return [] # cant move the elevator, dead end
    result = []
    for item in flooritems:
        # Take one item up or down, checking if both floors are still safe
        for newfloor in elevator_directions(node.floor):
            newnode = node.move_item(newfloor, item)
            if not safe(newnode.floors[node.floor]) or \
               not safe(newnode.floors[newfloor]): continue
            result.append((newnode, 1))
    if len(flooritems) == 1: return result
    
    for i1, i2 in combinations(flooritems, 2):
        for newfloor in elevator_directions(node.floor):
            newnode = node.move_item(newfloor, i1, False)
            newnode = newnode.move_item(newfloor, i2)
            if not safe(newnode.floors[node.floor]) or \
               not safe(newnode.floors[newfloor]): continue
            result.append((newnode, 1))
    return result

result = solve_maze(start, lambda x: x == end, adjacent)   

  0%|          | 0/1 [00:00<?, ?it/s]

Final path length: 37


In [ ]:
items = ['SG', 'SM', 'PG', 'PM', 'TM', 'TG', 'RG', 'RM', 'CG', 'CM', 'DG', 'DM', 'EG', 'EM']
start = Node(
    0,
    (frozenset(['SG', 'SM', 'PG', 'PM', 'DG', 'DM', 'EG', 'EM']),
    frozenset(['TG', 'RG', 'RM', 'CG', 'CM']),
    frozenset(['TM']),
    e)
)
end = (3 , (e, e, e, frozenset(items)))

# It took 7 minutes but it got it done!
result = solve_maze(start, lambda x: x == end, adjacent, update_tqdm_every=10000)   

  0%|          | 0/1 [00:00<?, ?it/s]

Final path length: 61
